<a href="https://colab.research.google.com/github/sharifashik591/Natural-language-processing/blob/main/Movie_Name_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np


In [5]:
df=pd.read_csv('tmdb_5000_movies.csv')
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [6]:
df=df['original_title']

In [7]:
df.head()

0                                      Avatar
1    Pirates of the Caribbean: At World's End
2                                     Spectre
3                       The Dark Knight Rises
4                                 John Carter
Name: original_title, dtype: object

In [8]:
movie_name=df.to_list()

In [ ]:
movie_name

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense,Embedding,LSTM



In [11]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(movie_name)
sequence=tokenizer.texts_to_sequences(movie_name)



In [12]:
sequence[:10]

[[1564],
 [210, 2, 1, 431, 47, 432, 72],
 [1565],
 [1, 52, 211, 1566],
 [212, 601],
 [213, 8, 21],
 [1567],
 [902, 146, 2, 1568],
 [110, 214, 4, 1, 433, 53, 147],
 [173, 340, 261, 85, 2, 903]]

In [ ]:
tokenizer.word_index

In [14]:
x=[]
y=[]

drop_word=0

for i in sequence:
  if len(i)>1:
    for index in range(1, len(i)):
      x.append(i[:index])
      y.append(i[index])
  else:
    drop_word += 1

print('Drop word sum= ', drop_word)




Drop word sum=  1003


In [15]:
print(x[:20])
print('\n\n\n')
print(y[:20])

[[210], [210, 2], [210, 2, 1], [210, 2, 1, 431], [210, 2, 1, 431, 47], [210, 2, 1, 431, 47, 432], [1], [1, 52], [1, 52, 211], [212], [213], [213, 8], [902], [902, 146], [902, 146, 2], [110], [110, 214], [110, 214, 4], [110, 214, 4, 1], [110, 214, 4, 1, 433]]




[2, 1, 431, 47, 432, 72, 52, 211, 1566, 601, 8, 21, 146, 2, 1568, 214, 4, 1, 433, 53]


In [16]:
x=pad_sequences(x)

In [17]:
x.shape

(8483, 14)

In [18]:
y=to_categorical(y)

In [19]:
y.shape

(8483, 5045)

In [20]:
voc_size=len(tokenizer.word_index) +1
voc_size

5045

In [21]:
model= tf.keras.Sequential([
                      Embedding(voc_size,14),
                      LSTM(100,return_sequences=True),      
                      LSTM(100),
                      Dense(100, activation='relu'),      
                      Dense(voc_size,activation='softmax')   
])

In [22]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 14)          70630     
_________________________________________________________________
lstm (LSTM)                  (None, None, 100)         46000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 5045)              509545    
Total params: 716,675
Trainable params: 716,675
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x,y, epochs=100, batch_size=16)

In [25]:
model.save('h5.h5')

In [26]:
voc_array=np.array(list(tokenizer.word_index.keys()))

In [27]:
voc_array

array(['the', 'of', 'a', ..., 'signed', 'sealed', 'delivered'],
      dtype='<U14')

In [32]:
def make_prediction(text,n_word):
  for i in range(n_word):
    text_tokenize=tokenizer.texts_to_sequences([text])
    text_padding=pad_sequences(text_tokenize, maxlen=14)
    prediction=np.squeeze(np.argmax(model.predict(text_padding),axis=-1))
    prediction=str(voc_array[prediction -1 ])
    text +=" "+ prediction
  return text

In [38]:
make_prediction('Harry',4)

'Harry potter and the chipmunks'